In [1]:
import numpy as np
import math
import csv 
import stripy

rng = np.random.default_rng()

In [2]:
def cart2sph(x, y, z):
    r = np.sqrt(np.power(x, 2) + np.power(y, 2) + np.power(z, 2))
    colat = np.arctan2(np.sqrt(np.power(x, 2) + np.power(y, 2)), z)
    lon = np.arctan2(y, x)
    return r, colat, lon

def latlon(x, y, z): # returns latitude and longitude in degrees 
    r, colat, lon = cart2sph(x, y, z)
    return lon, math.pi / 2 - colat

def sph2cart(r, colat, lon):
    x = r * np.sin(colat) * np.cos(lon)
    y = r * np.sin(colat) * np.sin(lon)
    z = r * np.cos(colat)
    return x, y, z

def gcdist(p1, p2, radius):
    s = np.dot(p1, p2) / (np.linalg.norm(p1) * np.linalg.norm(p2))
    theta = np.arccos(np.minimum(np.maximum(s, -1), 1))
    return radius * theta

In [15]:
fmesh = stripy.spherical_meshes.icosahedral_mesh(refinement_levels=7, include_face_points=False)
point_count = fmesh.npoints
points = np.zeros((point_count, 3))
values = np.zeros(point_count)
latlons = np.zeros((point_count, 2))
areas = 4 * math.pi * np.ones(point_count) / point_count
lon0 = 0
lat0 = -math.pi / 20
for i in range(point_count):
#     lon = rng.random() * 2 * math.pi
    lon = fmesh.lons[i]
    lat = math.pi / 2 - fmesh.lats[i]
#     lat = np.arccos(2 * rng.random() - 1)
    x, y, z = sph2cart(1, lat, lon)
    lat = math.pi / 2 - lat
    points[i,:] = [x, y, z]
    latlons[i,:] = [lat, lon]
    values[i] = 4 * math.pi * np.exp(-16 * np.square(gcdist(sph2cart(1, math.pi / 2 - lat, lon), sph2cart(1, math.pi / 2 - lat0, lon0), 1))) # g vor
#     values[i] = 2 * math.pi / 7 * np.sin(lat) + 30 * np.sin(lat) * np.power(np.cos(lat), 4) * np.cos(4 * lon) # RH wavenumber 4
#     values[i] = 4 * math.pi / 35 * np.sin(lat) + 72 * np.sin(lat) * np.power(np.cos(lat), 7) * np.cos(7 * lon) # RH wavenumber 7
total_vor = np.sum(areas * values)
for i in range(point_count):
    values[i] -= total_vor / (4 * math.pi)

In [16]:
values

array([-0.18342118, -0.1570059 , -0.17359317, ..., -0.17196552,
       -0.1659572 , -0.17422039])

In [36]:
with open('points.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(point_count):
        spamwriter.writerow([points[i][0], points[i][1], points[i][2], values[i]])

In [37]:
with open('tris.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(np.shape(fmesh.simplices)[0]):
        tris = fmesh.simplices[i]
        spamwriter.writerow([tris[0], tris[1], tris[2]])

In [38]:
with open('vert_tris.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(point_count):
        tris = fmesh.identify_vertex_triangles(i)
        if (np.size(tris) == 5):
            spamwriter.writerow([tris[0], tris[1], tris[2], tris[3], tris[4]])
        else:
            spamwriter.writerow([tris[0], tris[1], tris[2], tris[3], tris[4], tris[5]])

In [39]:
with open('vert_tri_count.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(point_count):
        count = np.size(fmesh.identify_vertex_triangles(i))
        spamwriter.writerow([count])

In [40]:
fmesh.npoints

10242

In [16]:
np.shape(fmesh.simplices)[0]

1280

In [24]:
np.amin(fmesh.lats)

-1.5707963267948966

In [3]:
fmesh = stripy.spherical_meshes.icosahedral_mesh(refinement_levels=3, include_face_points=False)
fmesh.npoints

642

In [8]:
np.shape(fmesh.simplices)[0]

1280

In [34]:
fmesh.identify_vertex_triangles(2560)

array([ 816,  821, 3416, 3417, 4756, 4841])

In [27]:
tris = fmesh.identify_vertex_triangles(2560)

In [35]:
np.size(tris)

6

In [41]:
test = np.zeros(2562)
for i in range(2562):
    test[i] = np.size(fmesh.identify_vertex_triangles(i))

In [44]:
test.max()

6.0

In [17]:
np.sum(values * areas)

-3.99650473305313e-16